In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
HW = 'humpback-whale-identification'
TRAIN = '../input/humpback-whale-identification/train/'
TEST = '../input/humpback-whale-identification/test/'
LABELS = '../input/humpback-whale-identification/train.csv'
SAMPLE_SUB = '../input/humpback-whale-identification/sample_submission.csv'
BBOX = '../input/metadata/bounding_boxes.csv'

whales_bbox = pd.read_csv(BBOX).set_index('Image')

train = pd.read_csv(LABELS)
print("With new_whale:")
train.head()

In [ ]:
MODEL_F = 'Model_ResNet50_30epocs.h5'
WEIGHTS_F = 'Weights_ResNet50_30epocs.h5'
MODEL = '../input/whales-draft/'+ MODEL_F
WEIGHTS = '../input/whales-draft/'+ WEIGHTS_F

In [ ]:
train.describe()

In [ ]:
import random 
from IPython.display import Image
# whales_train_modified.describe()

#show sample image
name = random.choice(train['Image'])
print(name)
Image(filename=TRAIN+name)

In [ ]:
train_images = train.set_index('Image')
new_whale_train = train_images[train_images.Id == "new_whale"]  # only new_whale dataset
# whales_train = train_images[~(train_images.Id == "new_whale")]  # no new_whale dataset, used for training
criteria = train['Id'] != 'new_whale'
whales_train = train[criteria]
    
print("Without new_whale:")
whales_train.head()

In [ ]:
unique_labels = np.unique(whales_train.Id.values)

In [ ]:
whales_train.describe()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model

import keras.backend as K
from keras.models import Sequential
from PIL import Image
import gc
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

%matplotlib inline

In [ ]:
def prepareImages(data, m, dataset):
    print("Preparing images")
    X_train = np.zeros((m, 100, 100, 3))
    count = 0
    
    for fig in data['Image']:          
        filepath = "../input/"+dataset+"/"+fig
        img = image.load_img(filepath)
        img = img.convert(mode="RGB")
        
        # load bounding box
        bbox = whales_bbox.loc[fig]
        x0, y0, x1, y1 = bbox['x0'], bbox['y0'], bbox['x1'], bbox['y1']
        
        # crop according to bounding box
        if not (x0 >= x1 or y0 >= y1):
            img = img.crop((x0, y0, x1, y1))
        #load images into images of size 100x100x3
        img = img.resize((100, 100))
        x = image.img_to_array(img)
        x = preprocess_input(x)

        X_train[count] = x
        if (count%500 == 0):
#             plt.imshow(img)
            print("Processing image: ", count+1, ", ", fig)
            
        count += 1
    
    return X_train

In [ ]:
def remove_new_whale():    
    labels_dict = dict()
    labels_list = []

    for i in range(len(unique_labels)):
        labels_dict[unique_labels[i]] = i
        labels_list.append(unique_labels[i])

    print("Number of classes: {}".format(len(unique_labels)))

    print(np.shape(labels_list))
    labels_list = np.array(labels_list)
    return labels_list, labels_dict

In [ ]:
labels_list, labels_dict = remove_new_whale()

In [ ]:
whales_train.Id = whales_train.Id.apply(lambda x: labels_dict[x])

In [ ]:
print(whales_train.head())

In [ ]:
def prepare_labels(y):
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    # print(integer_encoded)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
#     print(integer_encoded)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
#     print(onehot_encoded)

    y = onehot_encoded
    print(y.shape)
    return y, label_encoder

In [ ]:
y, label_encoder = prepare_labels(whales_train['Id'])
y.shape

In [ ]:
%matplotlib inline
X = prepareImages(whales_train, whales_train.shape[0], "humpback-whale-identification/train")
X /= 255

In [ ]:
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.losses import binary_crossentropy

CLASSES = 5004
    
# setup model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape = (100, 100, 3))

x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)
predictions = Dense(CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
   
# transfer learning
for layer in base_model.layers:
  layer.trainable = True
      
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# from keras.models import load_model

# # returns a compiled model
# # identical to the previous cell
# model = load_model(MODEL)
# print("Loaded model architecture from disk")

In [ ]:
print("Train set shape: "+ str(np.shape(X)))
print(np.shape(whales_train['Id']))
history = model.fit(X, y, epochs=30, batch_size=100, verbose=1)

In [ ]:
model.save(MODEL_F)
print("Saved model architecture to disk")
model.save_weights(WEIGHTS_F)
print("Saved model weights to disk")

In [ ]:
gc.collect()

In [ ]:
# model.load_weights(WEIGHTS)
# print("Loaded model weifhts from disk")

In [ ]:
plt.plot(history.history['acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
test = os.listdir(TEST)
print("Test set length: "+str(len(test)))

In [ ]:
col = ['Image']
test_df = pd.DataFrame(test, columns=col)
test_df['Id'] = ''

In [ ]:
X = prepareImages(test_df, test_df.shape[0], "humpback-whale-identification/test")
X /= 255

In [ ]:
predictions = model.predict(np.array(X), verbose=1)
print(np.shape(predictions))


np.save("test_files.npy", test)
np.save("one_hot_predss.npy", predictions)
np.save("one_hot_encoder.npy", label_encoder)

In [ ]:
print(labels_list[:7])
labels_with_new_whale = ['new_whale']
                         
for s in labels_list:
    labels_with_new_whale.append(s)

In [ ]:
def add_new_whale_to_predictions(preds):
    sorted_preds = np.sort(preds)
    avg_of_max_predictions = np.average(sorted_preds[:, -1:])
    print("Average of max probabilities column:" + str(avg_of_max_predictions))
    best_threshold = avg_of_max_predictions
    # print(np.shape(preds))
    shape_to_add = (np.shape(preds)[0], 1)
    
    # Add a column with the best threshold probability to the predictions
    column_to_add = np.zeros(shape_to_add) + best_threshold
    predictions_w_new_whale = np.concatenate([column_to_add, preds], axis=1)
    return predictions_w_new_whale

In [ ]:
def create_results_csv(preds, labels_with_new_whale, test_file_names, output_filename):
    sample_df = pd.read_csv(SAMPLE_SUB)
    sample_images = list(sample_df.Image)

    print(test_file_names[:7])
    pred_list = [[labels_with_new_whale[i] for i in p.argsort()[-5:][::-1]] for p in preds]

    pred_dic = dict((key, value) for (key, value) in zip(test_file_names, pred_list))
    pred_list_for_test = [' '.join(pred_dic[id]) for id in sample_images]

    print(np.shape(pred_list))
    print(np.shape(test_file_names))
    df = pd.DataFrame({'Image': sample_images, 'Id': pred_list_for_test})
    df.to_csv(output_filename, header=True, index=False)
    return df

In [ ]:
p = add_new_whale_to_predictions(predictions)
test_df = create_results_csv(p, labels_with_new_whale, test, "submission.csv")
print(test_df[:10])